# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
import logging
import json
import requests
import pandas as pd
import numpy as np
from numpy import array
import joblib
from sklearn.model_selection import train_test_split
import os
import azureml.core
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core.model import Model


In [2]:
#Environment Setup
from azureml.core import Environment
# set up the  environment
env = Environment.from_conda_specification(
        name='automl-env',
        file_path='./automl-env.yml'    
        )

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Automl'

experiment=Experiment(ws, experiment_name)

# data that will be used from this experiment was downloaded from Zindi Africa. It was designed by Data Science Nigeria
# .. for a hackathon. In the dataset we have various features of buildings and the goal is to predict
#.. whether a building will have an insurance claim over a given period of time
#.. the link is here: "https://zindi.africa/competitions/data-science-nigeria-2019-challenge-1-insurance-prediction/data"
# Get the data of Kaggle Titanic Dataset
key = "insurance data"
description_text = "Data from Zindi Africa Data hackathon"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/Ogbuchi-Ikechukwu/Azure-ML-Nanodegree-Capstone/master/starter_file/train_data_cleaned.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)


data = dataset.to_pandas_dataframe()


data.describe()

,YearOfObservation,Insured_Period,Residential,Build_Paint_N,Build_Fence_N,Garden_V,Garden_O,Garden_T,Settlement2,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
count,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7160.000000,7092.000000,7160.000000
mean,2013.669553,0.909758,0.305447,0.248324,0.503911,0.495950,0.503073,0.000978,0.495810,1855.839944,2.186034,1825.078771,2.193156,50133.089679,0.228212
std,1.383769,0.239756,0.460629,0.432071,0.500020,0.500019,0.500025,0.031254,0.500017,2272.645786,0.940632,505.583685,2.549629,30365.982905,0.419709
min,2012.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.000000,0.997268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000,2.000000,1956.750000,0.000000,17300.000000,0.000000
50%,2013.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1050.000000,2.000000,1960.000000,1.000000,56121.000000,0.000000
75%,2015.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2250.000000,3.000000,1980.000000,4.000000,75118.000000,0.000000
max,2016.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,20940.000000,4.000000,2016.000000,11.000000,95607.000000,1.000000


In [6]:
data=data.dropna()

In [7]:
#creating compute cluster
compute_cluster_name = "ikecompute"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [8]:
y=data["Claim"]
data.pop("Claim")
x=data

In [9]:
from sklearn.model_selection import train_test_split
#create train test split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=32)
#join the train_x and train_y to create AUTOML training dataset
train_data=pd.concat([x_train,y_train], axis=1)

In [10]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(train_data).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='insurance', overwrite=True, show_progress=True)

 

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('insurance/train_data.csv'))
label = "Claim"

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [11]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "enable_early_stopping":True,
    "max_concurrent_iterations":5,
    "primary_metric": "accuracy",
    "iteration_timeout_minutes": 5,
    "featurization": 'auto',
    "verbosity": logging.INFO
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_cluster,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='Claim',
    n_cross_validations=4)

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
from azureml.core.run import Run
#Getting best run
best_run, fitted_model=remote_run.get_output()
print(best_run)
print(fitted_model)
best_run.get_tags()
print(best_run.properties['model_name'])

Run(Experiment: Automl,
Id: AutoML_6725d1ca-3db3-4e35-ad53-123dd00d0ff6_25,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.33789473684210525,
                                                                                                  min_weight_fraction_leaf=0.0,
       

In [ ]:
#TODO: Save the best model
import joblib
os.makdirs

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Running predictions on sample data

scoring_uri = aci_service.scoring_uri
# If the service is authenticated, set the key or token
primary,secondary = aci_service.get_keys()
key = primary

# Convert to JSON string
x_test_json = x_test.to_json(orient='records')
data = "{\"data\": " + x_test_json +"}"

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, data, headers=headers)
print(resp.text)
#resp = requests.post(aci_service.scoring_uri, data, headers=headers)

y_pred = json.loads(json.loads(resp.text))['result']

In [ ]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(test_y, y_pred)
print(matrix)
print('Overall accuracy of AutoML:', np.average(y_pred == test_y))

In [ ]:
#Getting logs from ACI
aci_service.get_logs()

In [ ]:
aci_service.delete()